In [17]:
#If running in Anaconda/Jupyter Notebook, create a new anaconda environment and install geopandas, otherwise it won't run

#Using NPS Lands Layer Package

import geopandas as gpd
import pandas as pd

    
ALLCRASHES = pd.read_csv(r"C:\Users\Christopher.Dettmer\Documents\TSP\Using New Data\IMARS_crash_expanded_8-25-22.csv")
ALLBOUNDARIES=gpd.read_file(r"C:\Users\Christopher.Dettmer\Documents\TSP\Spatial Join Files\nps_boundary.shp")

def NCR_Cleaner(ALLCRASHES):
    
    #This is the list of parks that (1) are not assigned the NACA label in the NPS Lands Boundary and
    #(2) also have crashes with a park and coordinate filled; may not be entirely complete but should cover
    #any parks that would be in the top 15 for the NCR
    
    nonNACAlist=["GWMP","ROCR","LINC","MANA","JEFM","CATO","PRWI","CLBA","THIS","COGA","FRDE","GREE",
                 "ANTI","PAAV","PISC","MONO","WHHO","FOWA","HAFE","ARHO","WOTR","MLKM","DDEM","CHOH",
                 "BEPA","LYBA","MALL","MABE","WWII","FOTH","WAMO","KOWA","FRDO","CAWO","VIVE","WWIM"]
    
    for crash in range(len(ALLCRASHES)):
        park=ALLCRASHES.iloc[crash][1]
        region=ALLCRASHES.iloc[crash][4]
        
        if park not in nonNACAlist and region=="NC":
            ALLCRASHES.iat[crash,1]="NACA"
            
    return ALLCRASHES


def crashChooser(ALLCRASHES_CLEAN,ALLBOUNDARIES,parkCode):
        
    #Take park crashes, turn into a dataframe with coords, change from geometric to projected coords for sjoin
    park_crashes_df=ALLCRASHES_CLEAN.loc[ALLCRASHES_CLEAN['Park']==parkCode]
    park_crashes=gpd.GeoDataFrame(park_crashes_df, geometry=gpd.points_from_xy(park_crashes_df.Longitude,park_crashes_df.Latitude))
    proj_park_crashes=park_crashes.set_crs(epsg=3857)
    
    return proj_park_crashes

def boundaryChooser(ALLCRASHES_CLEAN,ALLBOUNDARIES,parkCode):
    
    #Take park boundary(ies), change from geometric to projected coords for sjoin
    park_polygon=ALLBOUNDARIES.loc[ALLBOUNDARIES['UNIT_CODE']==parkCode]
    proj_park_polygon=park_polygon.set_crs(epsg=3857,allow_override=True)          
        
    return proj_park_polygon
    

def sjoin_0(proj_park_crashes_clean, proj_park_polygon):
    
    return gpd.sjoin(proj_park_polygon,proj_park_crashes_clean,how='left')


def sjoin_1(proj_park_crashes_clean, proj_park_polygon):
    
    #Take park boundary(ies) with projected coords, add buffer, then reformat to geodataseries
    park_polygon_1_buffer_geoseries=gpd.GeoSeries.buffer(proj_park_polygon,0.0145055773)
    park_polygon_1_buffer=gpd.GeoDataFrame(geometry=gpd.GeoSeries(park_polygon_1_buffer_geoseries))

    return gpd.sjoin(park_polygon_1_buffer,proj_park_crashes_clean,how='left')


def sjoin_10(proj_park_crashes_clean, proj_park_polygon):
    
    #Take park boundary(ies) with projected coords, add buffer, then reformat to geodataseries
    park_polygon_10_buffer_geoseries=gpd.GeoSeries.buffer(proj_park_polygon,0.1450557739)
    park_polygon_10_buffer=gpd.GeoDataFrame(geometry=gpd.GeoSeries(park_polygon_10_buffer_geoseries))

    return gpd.sjoin(park_polygon_10_buffer,proj_park_crashes_clean,how='left')


def sjoin_100(proj_park_crashes_clean, proj_park_polygon):
    
    #Take park boundary(ies) with projected coords, add buffer, then reformat to geodataseries
    park_polygon_100_buffer_geoseries=gpd.GeoSeries.buffer(proj_park_polygon,1.45055774)
    park_polygon_100_buffer=gpd.GeoDataFrame(geometry=gpd.GeoSeries(park_polygon_100_buffer_geoseries))

    return gpd.sjoin(park_polygon_100_buffer,proj_park_crashes_clean,how='left')


def noCoordsCleaner(proj_park_crashes):

    noCoords=0
    
    for crash in range(len(proj_park_crashes)):
        
        if pd.isnull(proj_park_crashes.iloc[crash][2])==True or pd.isnull(proj_park_crashes.iloc[crash][3])==True:    
            noCoords+=1
            #proj_park_crashes.drop(crash)
            
    return proj_park_crashes, noCoords
    

def calculations(proj_park_crashes, proj_park_polygon, outputDataFrame, output_df_park, output_df_region):
    
    proj_park_crashes_clean, noCoords=noCoordsCleaner(proj_park_crashes)
    
    within0=len(sjoin_0(proj_park_crashes_clean, proj_park_polygon))
    within1=len(sjoin_1(proj_park_crashes_clean, proj_park_polygon))
    within10=len(sjoin_10(proj_park_crashes_clean, proj_park_polygon))
    within100=len(sjoin_100(proj_park_crashes_clean, proj_park_polygon))
    
    #Unable to drop crashes without coordinates in noCoordsCleaner, receiving errors
    #Workaround by subtracting crashes without coordinates from crashes over 100 miles outside of park boundary
    
    totalCrashes=len(proj_park_crashes_clean)
    over100=totalCrashes-within100-noCoords
    over10=within100-within10
    over1=within10-within1
    over0=within1-within0
    inBoundary=within0
    
    outputDataFrame.loc[len(outputDataFrame.index)]=[output_df_park,output_df_region,inBoundary,over0,over1,over10,over100,noCoords,totalCrashes]
    
    return outputDataFrame
    
    
def main():
    
    outputDataFrame=pd.DataFrame(columns=["Park","Region","Within Boundary","<1mi Outside","1-10mi Outside",
                                          "10-100mi Outside",">100mi Outside","No Coordinates","Total Crashes"])
    
    ALLCRASHES_CLEAN=NCR_Cleaner(ALLCRASHES)
    
    for park in range(len(ALLBOUNDARIES)): #for every park in the full set of boundaries 
        
        parkCode=ALLBOUNDARIES.loc[park][1] #take individual park code
        
        proj_park_crashes=crashChooser(ALLCRASHES_CLEAN,ALLBOUNDARIES,parkCode) #select park-specific crashes
        proj_park_polygon=boundaryChooser(ALLCRASHES_CLEAN,ALLBOUNDARIES,parkCode) #select park-specific boundary(ies)
        
        output_df_park=proj_park_polygon.iloc[0][1] #select park code
        output_df_region=str(proj_park_polygon.iloc[0][6])+"R" #select region code
        
        #Some AKR parks are recorded twice in an input dataset, must not record duplicates
        
        duplicate=output_df_park in outputDataFrame["Park"].values
        if duplicate==False:     
        
            if len(proj_park_crashes)==0: #if no crashes in a park, don't do spatial join calcs and add 0s to output df
                
                outputDataFrame.loc[len(outputDataFrame.index)]=[output_df_park,output_df_region,0,0,0,0,0,0,0]
            
            else:
                
                outputDataFrame=calculations(proj_park_crashes, proj_park_polygon, outputDataFrame, output_df_park, output_df_region)
            
    #Output spreadsheet code here: ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    outputDataFrame.to_excel(r"C:\Users\Christopher.Dettmer\Documents\TSP\Spatial Join Files\8-25-22 Final Coordinate Stats and Charts.xlsx",
                             sheet_name="Output Data", index = False)

main()
